In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import kurtosis
from scipy.stats import skew
import seaborn as sns
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV,KFold

In [3]:
train1 = pd.read_csv(r"F:\Cdac_project\train.csv",dtype={'acoustic_data':np.int16, 'time_to_failure':np.float64},nrows=153600000)

In [5]:
train2 = pd.read_csv(r"F:\Cdac_project\train.csv",dtype={'acoustic_data':np.int16, 'time_to_failure':np.float64},skiprows=153600000,nrows=153600000,names=['acoustic_data','time_to_failure'])
#train2.columns['acoustic_data','time_to_failure']


In [6]:
train3 = pd.read_csv(r"F:\Cdac_project\train.csv",dtype={'acoustic_data':np.int16, 'time_to_failure':np.float64},skiprows=153600000*2,nrows=153600000,names=['acoustic_data','time_to_failure'])

In [7]:
train4 = pd.read_csv(r"F:\Cdac_project\train.csv",dtype={'acoustic_data':np.int16, 'time_to_failure':np.float64},skiprows=153600000*3,nrows=153600000,names=['acoustic_data','time_to_failure'])

In [8]:
rows = 150000
segments = int(np.floor(train1.shape[0] / rows))
print("Number of segments: ", segments)
print(type(segments))

X_train = pd.DataFrame(index = range(segments), dtype = np.float64, columns = ['ave', 'std', 'max', 'min','sum','skew','kurt','var','quantile_1','quantile_2','quantile_3','median'])
y_train = pd.DataFrame(index = range(segments), dtype= np.float64, columns = ['time_to_failure'])

Number of segments:  1024
<class 'int'>


In [9]:
def add_features(data,n):
    for i in range(segments):
        # Selecting the rows from each segment in the data.
        seg = data.iloc[i*rows : (i*rows)+rows]
        i = i+1024*n
        
        x = seg['acoustic_data'].values
        y = seg['time_to_failure'].values
        #y = pd.DataFrame(y)
    
        y_train.loc[i,'time_to_failure'] = y.mean()
        #print(y_train.head())
        
        X_train.loc[i, 'ave'] = x.mean()
        X_train.loc[i, 'std'] = x.std()
        X_train.loc[i, 'max'] = x.max()
        X_train.loc[i, 'min'] = x.min()
        X_train.loc[i, 'sum'] = x.sum()
        X_train.loc[i, 'skew'] = skew(x)
        X_train.loc[i, 'kurt'] = kurtosis(x)
        X_train.loc[i,'var'] = x.var()
        X_train.loc[i,'quantile_1'] = seg['acoustic_data'].quantile(0.25)
        X_train.loc[i,'quantile_2'] = seg['acoustic_data'].quantile(0.50)
        X_train.loc[i,'quantile_3'] = seg['acoustic_data'].quantile(0.75)
        X_train.loc[i,'median'] = seg['acoustic_data'].median()
        #print(X_train.head())

#add_features(train1,0)
#print(X_train.shape)
#print(y_train.shape)

In [10]:
add_features(train1,0)

In [11]:
print(X_train.shape)
print(y_train.shape)

(1024, 12)
(1024, 1)


In [39]:
X_train.head(10)

,ave,std,max,min,sum,skew,kurt,var,quantile_1,quantile_2,quantile_3,median
0,4.884113,5.101089,104.0,-98.0,732617.0,-0.024061,33.661319,26.021110,3.0,5.0,7.0,5.0
1,4.725767,6.588802,181.0,-154.0,708865.0,0.390557,98.755185,43.412309,2.0,5.0,7.0,5.0
2,4.906393,6.967374,140.0,-106.0,735959.0,0.217388,33.554053,48.544298,2.0,5.0,7.0,5.0
3,4.902240,6.922282,197.0,-199.0,735336.0,0.757270,116.544247,47.917990,2.0,5.0,7.0,5.0
4,4.908720,7.301086,145.0,-126.0,736308.0,0.064530,52.976099,53.305855,2.0,5.0,7.0,5.0
5,4.913513,5.434093,142.0,-144.0,737027.0,-0.100696,50.213433,29.529367,3.0,5.0,7.0,5.0
6,4.855660,5.687804,120.0,-78.0,728349.0,0.208808,23.172192,32.351119,2.0,5.0,7.0,5.0
7,4.505427,5.854492,139.0,-134.0,675814.0,-0.176331,52.386951,34.275077,2.0,5.0,7.0,5.0
8,4.717833,7.789617,168.0,-156.0,707675.0,-0.160164,65.358042,60.678135,2.0,5.0,7.0,5.0
9,4.730960,6.890436,152.0,-126.0,709644.0,0.150777,53.758375,47.478111,2.0,5.0,7.0,5.0


In [41]:
y_train.head(10)

,time_to_failure
0,1.450182
1,1.411224
2,1.372258
3,1.333291
4,1.294332
5,1.255364
6,1.216402
7,1.177440
8,1.138470
9,1.099507


In [13]:
print(X_train.isna().sum().any())

False


In [14]:
add_features(train2,1)

In [15]:
add_features(train3,2)

In [16]:
add_features(train4,3)

In [17]:
print(X_train.shape)
print(y_train.shape)

(4096, 12)
(4096, 1)


In [18]:
X_train.isna().sum().any()

False

In [19]:
submission = pd.read_csv(r"F:\Cdac_project\sample_submission.csv", index_col='seg_id')
submission.head()

,time_to_failure
seg_id,
seg_00030f,0
seg_0012b5,0
seg_00184e,0
seg_003339,0
seg_0042cc,0


In [20]:
X_test = pd.DataFrame(columns=X_train.columns, dtype=np.float64, index=submission.index)

In [24]:
for seg_id in X_test.index:
    seg = pd.read_csv("F:/Cdac_project/test/" + seg_id + ".csv")
    
    x = seg['acoustic_data'].values
    
    X_test.loc[seg_id, 'ave'] = x.mean()
    X_test.loc[seg_id, 'std'] = x.std()
    X_test.loc[seg_id, 'max'] = x.max()
    X_test.loc[seg_id, 'min'] = x.min()
    X_test.loc[seg_id, 'sum'] = x.sum()
    X_test.loc[seg_id, 'skew'] = skew(x)
    X_test.loc[seg_id, 'kurt'] = kurtosis(x)
    X_test.loc[seg_id,'var'] = x.var()
    X_test.loc[seg_id,'quantile_1'] = seg['acoustic_data'].quantile(0.25)
    X_test.loc[seg_id,'quantile_2'] = seg['acoustic_data'].quantile(0.50)
    X_test.loc[seg_id,'quantile_3'] = seg['acoustic_data'].quantile(0.75)
    X_test.loc[seg_id,'median'] = seg['acoustic_data'].median()

In [25]:
print(X_test)
X_test.isna().sum().any()

                 ave        std    max    min       sum      skew        kurt  \
seg_id                                                                          
seg_00030f  4.491780   4.893673  115.0  -75.0  673767.0  0.327904   28.836567   
seg_0012b5  4.171153   5.922820  152.0 -140.0  625673.0  0.295705   56.217041   
seg_00184e  4.610260   6.946967  248.0 -193.0  691539.0  0.428684  162.112840   
seg_003339  4.531473   4.114133   85.0  -93.0  679721.0  0.061889   41.240413   
seg_0042cc  4.128340   5.797144  177.0 -147.0  619251.0  0.073898   79.537016   
...              ...        ...    ...    ...       ...       ...         ...   
seg_ff4236  4.095773   5.082426   71.0  -50.0  614366.0 -0.001993   12.280456   
seg_ff7478  4.138493   6.031576  164.0 -208.0  620774.0 -0.843307  132.557911   
seg_ff79d9  3.748587   9.467500  310.0 -303.0  562288.0 -0.472015  202.478046   
seg_ffbd6a  4.341687  13.199310  362.0 -319.0  651253.0  0.057967  109.228711   
seg_ffe7cc  4.199720   3.573

False

In [39]:
kfold=KFold(n_splits=5,random_state=2020)

depth_range = [12,16,15,17,18]
minsplit_range = [5,10,20,25,30,3]
minleaf_range = [5,10,15,3]

parameters = dict(max_depth=depth_range,
                  min_samples_split=minsplit_range, 
                  min_samples_leaf=minleaf_range)
rfr = RandomForestRegressor(oob_score=True,random_state=2020)

cv = GridSearchCV(rfr,param_grid=parameters,cv=kfold,scoring='neg_mean_absolute_error',verbose=3)
y_train = y_train['time_to_failure']

F:\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


In [40]:
cv.fit(X_train,y_train)

Fitting 5 folds for each of 120 candidates, totalling 600 fits
[CV] max_depth=12, min_samples_leaf=5, min_samples_split=5 ...........


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  max_depth=12, min_samples_leaf=5, min_samples_split=5, score=-2.246, total=   1.0s
[CV] max_depth=12, min_samples_leaf=5, min_samples_split=5 ...........


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.9s remaining:    0.0s


[CV]  max_depth=12, min_samples_leaf=5, min_samples_split=5, score=-2.421, total=   1.0s
[CV] max_depth=12, min_samples_leaf=5, min_samples_split=5 ...........


...

[CV]  max_depth=17, min_samples_leaf=3, min_samples_split=20, score=-2.419, total=   1.1s
[CV] max_depth=17, min_samples_leaf=3, min_samples_split=20 ..........
[CV]  max_depth=17, min_samples_leaf=3, min_samples_split=20, score=-2.723, total=   1.1s
[CV] max_depth=17, min_samples_leaf=3, min_samples_split=20 ..........
[CV]  max_depth=17, min_samples_leaf=3, min_samples_split=20, score=-1.615, total=   1.1s
[CV] max_depth=17, min_samples_leaf=3, min_samples_split=20 ..........
[CV]  max_depth=17, min_samples_leaf=3, min_samples_split=20, score=-2.654, total=   1.1s
[CV] max_depth=17, min_samples_leaf=3, min_samples_split=25 ..........
[CV]  max_depth=17, min_samples_leaf=3, min_samples_split=25, score=-2.235, total=   1.0s
[CV] max_depth=17, min_samples_leaf=3, min_samples_split=25 ..........
[CV]  max_depth=17, min_samples_leaf=3, min_samples_split=25, score=-2.415, total=   1.0s
[CV] max_depth=17, min_samples_leaf=3, min_samples_split=25 ..........
[CV]  max_depth=17, min_samples_le

[CV]  max_depth=18, min_samples_leaf=10, min_samples_split=5, score=-2.714, total=   0.9s
[CV] max_depth=18, min_samples_leaf=10, min_samples_split=5 ..........
[CV]  max_depth=18, min_samples_leaf=10, min_samples_split=5, score=-1.603, total=   0.9s
[CV] max_depth=18, min_samples_leaf=10, min_samples_split=5 ..........
[CV]  max_depth=18, min_samples_leaf=10, min_samples_split=5, score=-2.629, total=   0.9s
[CV] max_depth=18, min_samples_leaf=10, min_samples_split=10 .........
[CV]  max_depth=18, min_samples_leaf=10, min_samples_split=10, score=-2.253, total=   0.9s
[CV] max_depth=18, min_samples_leaf=10, min_samples_split=10 .........
[CV]  max_depth=18, min_samples_leaf=10, min_samples_split=10, score=-2.407, total=   0.9s
[CV] max_depth=18, min_samples_leaf=10, min_samples_split=10 .........
[CV]  max_depth=18, min_samples_leaf=10, min_samples_split=10, score=-2.714, total=   1.0s
[CV] max_depth=18, min_samples_leaf=10, min_samples_split=10 .........
[CV]  max_depth=18, min_samples

[CV]  max_depth=18, min_samples_leaf=15, min_samples_split=30, score=-1.599, total=   0.8s
[CV] max_depth=18, min_samples_leaf=15, min_samples_split=30 .........
[CV]  max_depth=18, min_samples_leaf=15, min_samples_split=30, score=-2.618, total=   0.8s
[CV] max_depth=18, min_samples_leaf=15, min_samples_split=3 ..........
[CV]  max_depth=18, min_samples_leaf=15, min_samples_split=3, score=-2.244, total=   0.9s
[CV] max_depth=18, min_samples_leaf=15, min_samples_split=3 ..........
[CV]  max_depth=18, min_samples_leaf=15, min_samples_split=3, score=-2.391, total=   0.8s
[CV] max_depth=18, min_samples_leaf=15, min_samples_split=3 ..........
[CV]  max_depth=18, min_samples_leaf=15, min_samples_split=3, score=-2.708, total=   0.9s
[CV] max_depth=18, min_samples_leaf=15, min_samples_split=3 ..........
[CV]  max_depth=18, min_samples_leaf=15, min_samples_split=3, score=-1.599, total=   0.8s
[CV] max_depth=18, min_samples_leaf=15, min_samples_split=3 ..........
[CV]  max_depth=18, min_samples_

[Parallel(n_jobs=1)]: Done 600 out of 600 | elapsed:  9.7min finished


GridSearchCV(cv=KFold(n_splits=5, random_state=2020, shuffle=False),
             error_score=nan,
             estimator=RandomForestRegressor(bootstrap=True, ccp_alpha=0.0,
                                             criterion='mse', max_depth=None,
                                             max_features='auto',
                                             max_leaf_nodes=None,
                                             max_samples=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             n_estimators=100, n_jobs=None,
                                             oob_score=True, random_state=2020,
                                             ver

In [41]:
print(cv.best_score_)
print(cv.best_params_)
best1=cv.best_estimator_

-2.3117282973989406
{'max_depth': 12, 'min_samples_leaf': 15, 'min_samples_split': 5}


In [42]:
y_pred1=best.predict(X_test)
print(y_pred1)
type(y_pred1)


[ 5.0663359   3.73554437  4.357106   ...  4.42473661  2.06196901
 10.43796878]


numpy.ndarray

In [43]:
y_pred1 = pd.DataFrame(y_pred)
y_pred1.shape
y_pred1.head()

,0
0,5.066336
1,3.735544
2,4.357106
3,8.497970
4,6.092020


In [44]:
y_pred1.to_csv(r"F:\project\sample1.csv",index=False)

# XGB

In [28]:
from xgboost import XGBRegressor
lr_range = [0.001, 0.01, 0.1, 0.2,0.25, 0.3]
n_est_range = [10,20,30,50,100]
md_range = [2,4,6,8,10]

parameters = dict(learning_rate=lr_range,
                  n_estimators=n_est_range,
                  max_depth=md_range)

from sklearn.model_selection import GridSearchCV
clf = XGBRegressor(random_state=1211,silent=True)
cv1 = GridSearchCV(clf, param_grid=parameters,
                  cv=5,scoring='neg_mean_absolute_error',verbose=3)
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [29]:
cv1.fit(X_train,y_train)

Fitting 5 folds for each of 150 candidates, totalling 750 fits
[CV] learning_rate=0.001, max_depth=2, n_estimators=10 ...............
[CV]  learning_rate=0.001, max_depth=2, n_estimators=10, score=-5.704, total=   0.0s
[CV] learning_rate=0.001, max_depth=2, n_estimators=10 ...............
[CV]  learning_rate=0.001, max_depth=2, n_estimators=10, score=-4.071, total=   0.0s
[CV] learning_rate=0.001, max_depth=2, n_estimators=10 ...............
[CV]  learning_rate=0.001, max_depth=2, n_estimators=10, score=-5.917, total=   0.0s
[CV] learning_rate=0.001, max_depth=2, n_estimators=10 ...............
[CV]  learning_rate=0.001, max_depth=2, n_estimators=10, score=-4.673, total=   0.0s
[CV] learning_rate=0.001, max_depth=2, n_estimators=10 ...............
[CV]  learning_rate=0.001, max_depth=2, n_estimators=10, score=-5.396, total=   0.0s
[CV] learning_rate=0.001, max_depth=2, n_estimators=20 ...............
[CV]  learning_rate=0.001, max_depth=2, n_estimators=20, score=-5.658, total=   0.0s
[

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s


[CV]  learning_rate=0.001, max_depth=2, n_estimators=20, score=-5.870, total=   0.0s
[CV] learning_rate=0.001, max_depth=2, n_estimators=20 ...............
[CV]  learning_rate=0.001, max_depth=2, n_estimators=20, score=-4.626, total=   0.0s
[CV] learning_rate=0.001, max_depth=2, n_estimators=20 ...............
[CV]  learning_rate=0.001, max_depth=2, n_estimators=20, score=-5.351, total=   0.0s
[CV] learning_rate=0.001, max_depth=2, n_estimators=30 ...............
[CV]  learning_rate=0.001, max_depth=2, n_estimators=30, score=-5.612, total=   0.0s
[CV] learning_rate=0.001, max_depth=2, n_estimators=30 ...............
[CV]  learning_rate=0.001, max_depth=2, n_estimators=30, score=-3.983, total=   0.0s
[CV] learning_rate=0.001, max_depth=2, n_estimators=30 ...............
[CV]  learning_rate=0.001, max_depth=2, n_estimators=30, score=-5.824, total=   0.0s
[CV] learning_rate=0.001, max_depth=2, n_estimators=30 ...............
[CV]  learning_rate=0.001, max_depth=2, n_estimators=30, score=-

[CV]  learning_rate=0.001, max_depth=6, n_estimators=30, score=-5.618, total=   0.1s
[CV] learning_rate=0.001, max_depth=6, n_estimators=30 ...............
[CV]  learning_rate=0.001, max_depth=6, n_estimators=30, score=-3.989, total=   0.1s
[CV] learning_rate=0.001, max_depth=6, n_estimators=30 ...............
[CV]  learning_rate=0.001, max_depth=6, n_estimators=30, score=-5.825, total=   0.1s
[CV] learning_rate=0.001, max_depth=6, n_estimators=30 ...............
[CV]  learning_rate=0.001, max_depth=6, n_estimators=30, score=-4.582, total=   0.1s
[CV] learning_rate=0.001, max_depth=6, n_estimators=30 ...............
[CV]  learning_rate=0.001, max_depth=6, n_estimators=30, score=-5.305, total=   0.1s
[CV] learning_rate=0.001, max_depth=6, n_estimators=50 ...............
[CV]  learning_rate=0.001, max_depth=6, n_estimators=50, score=-5.532, total=   0.2s
[CV] learning_rate=0.001, max_depth=6, n_estimators=50 ...............
[CV]  learning_rate=0.001, max_depth=6, n_estimators=50, score=-

[CV]  learning_rate=0.001, max_depth=10, n_estimators=30, score=-4.587, total=   0.2s
[CV] learning_rate=0.001, max_depth=10, n_estimators=30 ..............
[CV]  learning_rate=0.001, max_depth=10, n_estimators=30, score=-5.309, total=   0.2s
[CV] learning_rate=0.001, max_depth=10, n_estimators=50 ..............
[CV]  learning_rate=0.001, max_depth=10, n_estimators=50, score=-5.541, total=   0.3s
[CV] learning_rate=0.001, max_depth=10, n_estimators=50 ..............
[CV]  learning_rate=0.001, max_depth=10, n_estimators=50, score=-3.912, total=   0.3s
[CV] learning_rate=0.001, max_depth=10, n_estimators=50 ..............
[CV]  learning_rate=0.001, max_depth=10, n_estimators=50, score=-5.738, total=   0.3s
[CV] learning_rate=0.001, max_depth=10, n_estimators=50 ..............
[CV]  learning_rate=0.001, max_depth=10, n_estimators=50, score=-4.500, total=   0.3s
[CV] learning_rate=0.001, max_depth=10, n_estimators=50 ..............
[CV]  learning_rate=0.001, max_depth=10, n_estimators=50, 

...

[CV]  learning_rate=0.25, max_depth=8, n_estimators=100, score=-1.833, total=   0.4s
[CV] learning_rate=0.25, max_depth=8, n_estimators=100 ...............
[CV]  learning_rate=0.25, max_depth=8, n_estimators=100, score=-2.735, total=   0.4s
[CV] learning_rate=0.25, max_depth=10, n_estimators=10 ...............
[CV]  learning_rate=0.25, max_depth=10, n_estimators=10, score=-2.426, total=   0.1s
[CV] learning_rate=0.25, max_depth=10, n_estimators=10 ...............
[CV]  learning_rate=0.25, max_depth=10, n_estimators=10, score=-2.381, total=   0.1s
[CV] learning_rate=0.25, max_depth=10, n_estimators=10 ...............
[CV]  learning_rate=0.25, max_depth=10, n_estimators=10, score=-2.831, total=   0.1s
[CV] learning_rate=0.25, max_depth=10, n_estimators=10 ...............
[CV]  learning_rate=0.25, max_depth=10, n_estimators=10, score=-1.727, total=   0.1s
[CV] learning_rate=0.25, max_depth=10, n_estimators=10 ...............
[CV]  learning_rate=0.25, max_depth=10, n_estimators=10, score=-

[CV]  learning_rate=0.3, max_depth=4, n_estimators=10, score=-2.679, total=   0.0s
[CV] learning_rate=0.3, max_depth=4, n_estimators=10 .................
[CV]  learning_rate=0.3, max_depth=4, n_estimators=10, score=-1.609, total=   0.0s
[CV] learning_rate=0.3, max_depth=4, n_estimators=10 .................
[CV]  learning_rate=0.3, max_depth=4, n_estimators=10, score=-2.587, total=   0.0s
[CV] learning_rate=0.3, max_depth=4, n_estimators=20 .................
[CV]  learning_rate=0.3, max_depth=4, n_estimators=20, score=-2.276, total=   0.1s
[CV] learning_rate=0.3, max_depth=4, n_estimators=20 .................
[CV]  learning_rate=0.3, max_depth=4, n_estimators=20, score=-2.427, total=   0.0s
[CV] learning_rate=0.3, max_depth=4, n_estimators=20 .................
[CV]  learning_rate=0.3, max_depth=4, n_estimators=20, score=-2.704, total=   0.0s
[CV] learning_rate=0.3, max_depth=4, n_estimators=20 .................
[CV]  learning_rate=0.3, max_depth=4, n_estimators=20, score=-1.602, total= 

[CV]  learning_rate=0.3, max_depth=8, n_estimators=20, score=-2.511, total=   0.1s
[CV] learning_rate=0.3, max_depth=8, n_estimators=20 .................
[CV]  learning_rate=0.3, max_depth=8, n_estimators=20, score=-2.811, total=   0.1s
[CV] learning_rate=0.3, max_depth=8, n_estimators=20 .................
[CV]  learning_rate=0.3, max_depth=8, n_estimators=20, score=-1.751, total=   0.1s
[CV] learning_rate=0.3, max_depth=8, n_estimators=20 .................
[CV]  learning_rate=0.3, max_depth=8, n_estimators=20, score=-2.658, total=   0.1s
[CV] learning_rate=0.3, max_depth=8, n_estimators=30 .................
[CV]  learning_rate=0.3, max_depth=8, n_estimators=30, score=-2.323, total=   0.1s
[CV] learning_rate=0.3, max_depth=8, n_estimators=30 .................
[CV]  learning_rate=0.3, max_depth=8, n_estimators=30, score=-2.527, total=   0.1s
[CV] learning_rate=0.3, max_depth=8, n_estimators=30 .................
[CV]  learning_rate=0.3, max_depth=8, n_estimators=30, score=-2.837, total= 

[Parallel(n_jobs=1)]: Done 750 out of 750 | elapsed:  1.8min finished


GridSearchCV(cv=5, error_score=nan,
             estimator=XGBRegressor(base_score=0.5, booster='gbtree',
                                    colsample_bylevel=1, colsample_bynode=1,
                                    colsample_bytree=1, gamma=0,
                                    importance_type='gain', learning_rate=0.1,
                                    max_delta_step=0, max_depth=3,
                                    min_child_weight=1, missing=None,
                                    n_estimators=100, n_jobs=1, nthread=None,
                                    objective='reg:linear', random_state=1211,
                                    reg_alpha=0, reg_lambda=1,
                                    scale_pos_weight=1, seed=None, silent=True,
                                    subsample=1, verbosity=1),
             iid='deprecated', n_jobs=None,
             param_grid={'learning_rate': [0.001, 0.01, 0.1, 0.2, 0.25, 0.3],
                         'max_depth': [2, 4, 6, 8, 

In [31]:
print(cv1.best_score_)

-2.275925424777679


In [32]:
print(cv1.best_params_)

{'learning_rate': 0.25, 'max_depth': 4, 'n_estimators': 10}


In [33]:
best2=cv1.best_estimator_

In [34]:
y_pred2=best2.predict(X_test)
print(y_pred2)
type(y_pred2)

[5.502506  4.145071  4.270093  ... 4.351365  2.2649877 9.03401  ]


numpy.ndarray

In [36]:
y_pred2 = pd.DataFrame(y_pred2)
y_pred2.shape
y_pred2.head()

,0
0,5.502506
1,4.145071
2,4.270093
3,7.812926
4,6.383760


In [38]:
y_pred2.to_csv(r"F:\Cdac_project\predicted_values\sample2.csv",index=False)

# h2o

In [42]:
t1=X_train

In [43]:
t1.head()

,ave,std,max,min,sum,skew,kurt,var,quantile_1,quantile_2,quantile_3,median
0,4.884113,5.101089,104.0,-98.0,732617.0,-0.024061,33.661319,26.021110,3.0,5.0,7.0,5.0
1,4.725767,6.588802,181.0,-154.0,708865.0,0.390557,98.755185,43.412309,2.0,5.0,7.0,5.0
2,4.906393,6.967374,140.0,-106.0,735959.0,0.217388,33.554053,48.544298,2.0,5.0,7.0,5.0
3,4.902240,6.922282,197.0,-199.0,735336.0,0.757270,116.544247,47.917990,2.0,5.0,7.0,5.0
4,4.908720,7.301086,145.0,-126.0,736308.0,0.064530,52.976099,53.305855,2.0,5.0,7.0,5.0


In [44]:
t1['time_to_failure']=y_train

In [45]:
t1.head()

,ave,std,max,min,sum,skew,kurt,var,quantile_1,quantile_2,quantile_3,median,time_to_failure
0,4.884113,5.101089,104.0,-98.0,732617.0,-0.024061,33.661319,26.021110,3.0,5.0,7.0,5.0,1.450182
1,4.725767,6.588802,181.0,-154.0,708865.0,0.390557,98.755185,43.412309,2.0,5.0,7.0,5.0,1.411224
2,4.906393,6.967374,140.0,-106.0,735959.0,0.217388,33.554053,48.544298,2.0,5.0,7.0,5.0,1.372258
3,4.902240,6.922282,197.0,-199.0,735336.0,0.757270,116.544247,47.917990,2.0,5.0,7.0,5.0,1.333291
4,4.908720,7.301086,145.0,-126.0,736308.0,0.064530,52.976099,53.305855,2.0,5.0,7.0,5.0,1.294332


In [46]:
import h2o
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
; Java HotSpot(TM) 64-Bit Server VM (build 25.121-b13, mixed mode)
  Starting server from F:\Python\anaconda\lib\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\dbda\AppData\Local\Temp\tmpt3go5wwu
  JVM stdout: C:\Users\dbda\AppData\Local\Temp\tmpt3go5wwu\h2o_dbda_started_from_python.out
  JVM stderr: C:\Users\dbda\AppData\Local\Temp\tmpt3go5wwu\h2o_dbda_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O cluster uptime:,05 secs
H2O cluster timezone:,+05:30
H2O data parsing timezone:,UTC
H2O cluster version:,3.28.0.1
H2O cluster version age:,1 month and 2 days
H2O cluster name:,H2O_from_python_dbda_jdo9sq
H2O cluster total nodes:,1
H2O cluster free memory:,3.531 Gb
H2O cluster total cores:,8
H2O cluster allowed cores:,8
H2O cluster status:,"accepting new members, healthy"


In [47]:
df=h2o.H2OFrame(t1)

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [48]:
y = 'time_to_failure'
x = df.col_names
x.remove(y)
print("Response = " + y)
print("Pridictors = " + str(x))

Response = time_to_failure
Pridictors = ['ave', 'std', 'max', 'min', 'sum', 'skew', 'kurt', 'var', 'quantile_1', 'quantile_2', 'quantile_3', 'median']


In [53]:
df['time_to_failure'] = df['time_to_failure'].asfactor()
df['time_to_failure'].levels()

H2OValueError: Only 'int' or 'string' are allowed for asfactor(), got time_to_failure:real 

In [52]:
train, test = df.split_frame(ratios=[.7])
print(df.shape)
print(train.shape)
#print(valid.shape)
print(test.shape)

(4096, 13)
(2834, 13)
(1262, 13)


In [68]:
from h2o.grid.grid_search import H2OGridSearch
from h2o.estimators import H2OXGBoostEstimator
#model = H2OXGBoostEstimator(distribution = "gaussian")


In [69]:
glm_params1 = {
      "ntrees" : 100
    , "learn_rate" : 0.02
    , "sample_rate" : 0.7
    , "col_sample_rate_per_tree" : 0.9
    , "min_rows" : 5
    , "seed": 4241
    , "score_tree_interval": 100
}

In [61]:
glm_params2 = {'learn_rate': np.linspace(0.0001,0.2,30).tolist(),
                'max_depth': np.arange(10,300,50).tolist()}

In [75]:
hyper_params = {'ntrees' : np.linspace(10, 1000, 1).tolist(),
                     'learn_rate' : np.linspace(0.0001, 0.2, 0.0001).tolist(),
                     'max_depth' : np.linspace(1, 20, 1).tolist(),
                     'sample_rate' : np.linspace(0.5, 1.0, 0.0001).tolist(),
                     'col_sample_rate' : np.linspace(0.2, 1.0, 0.0001).tolist()}

F:\Python\anaconda\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: object of type <class 'float'> cannot be safely interpreted as an integer.
  
F:\Python\anaconda\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: object of type <class 'float'> cannot be safely interpreted as an integer.
  after removing the cwd from sys.path.
F:\Python\anaconda\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: object of type <class 'float'> cannot be safely interpreted as an integer.
  """


In [76]:
h2o_glm = H2OXGBoostEstimator(distribution = "gaussian")

# Train and validate a cartesian grid of GBMs
gbm_grid1 = H2OGridSearch(model=h2o_glm,
                          grid_id='gbm_grid1',
                          hyper_params=hyper_params)
gbm_grid1.train(x=x, y=y,
                training_frame=df,
                seed=1)

# Get the grid results, sorted by validation MAE
gbm_gridperf1 = gbm_grid1.get_grid(sort_by="MAE",decreasing=False)
gbm_gridperf1

# Grab the top GBM model, chosen by validation MAE
best_gbm1 = gbm_gridperf1.models[0]

H2OResponseError: Server error water.exceptions.H2ONotFoundArgumentException:
  Error: POST /99/Grid/xgboost not found
  Request: POST /99/Grid/xgboost
    data: {'parallelism': '1', 'hyper_parameters': "{'ntrees': [10.0], 'learn_rate': [], 'max_depth': [1.0], 'sample_rate': [], 'col_sample_rate': []}", 'distribution': 'gaussian', 'seed': '1', 'training_frame': 'Key_Frame__upload_81a866b6c2a42adac84d91c4f6cb46d0.hex', 'response_column': 'time_to_failure', 'grid_id': 'gbm_grid1'}


In [77]:
from h2o.estimators.gbm import H2OGradientBoostingEstimator


In [79]:
from h2o.grid.grid_search import H2OGridSearch

# GBM hyperparameters
glm_params1 = {'learn_rate': np.linspace(0.0001,0.2,30).tolist(),
                'max_depth': np.arange(10,300,50).tolist()}

h2o_glm = H2OGradientBoostingEstimator(distribution = "gaussian")

# Train and validate a cartesian grid of GBMs
gbm_grid1 = H2OGridSearch(model=h2o_glm,
                          grid_id='gbm_grid1',
                          hyper_params=glm_params1)
gbm_grid1.train(x=x, y=y,
                training_frame=df,
                seed=1)

# Get the grid results, sorted by validation MAE
gbm_gridperf1 = gbm_grid1.get_grid(sort_by="mae",decreasing=False)
gbm_gridperf1

# Grab the top GBM model, chosen by validation MAE
best_gbm1 = gbm_gridperf1.models[0]

gbm Grid Build progress: |████████████████████████████████████████████████| 100%
Errors/Warnings building gridsearch model

Hyper-parameter: learn_rate, 0.12417586206896554
Hyper-parameter: max_depth, 10
failure_details: None
failure_stack_traces: water.Job$JobCancelledException
	at hex.tree.SharedTree$Driver.scoreAndBuildTrees(SharedTree.java:434)
	at hex.tree.SharedTree$Driver.computeImpl(SharedTree.java:344)
	at hex.ModelBuilder$Driver.compute2(ModelBuilder.java:239)
	at hex.ModelBuilder.trainModelNested(ModelBuilder.java:381)
	at hex.ModelBuilder$TrainModelNestedRunnable.run(ModelBuilder.java:416)
	at water.H2O.runOnH2ONode(H2O.java:1343)
	at water.H2O.runOnH2ONode(H2O.java:1332)
	at hex.ModelBuilder.trainModelNested(ModelBuilder.java:397)
	at hex.grid.GridSearch.buildModel(GridSearch.java:524)
	at hex.grid.GridSearch.gridSearch(GridSearch.java:352)
	at hex.grid.GridSearch.access$000(GridSearch.java:69)
	at hex.grid.GridSearch$1.compute2(GridSearch.java:141)
	at water.H2O$H2OCounte

TypeError: unsupported operand type(s) for +: 'NoneType' and 'str'

In [91]:
family = "gaussian"



# Some XGboost/GBM hyperparameters
hyper_params = {'learn_rate': np.linspace(0.0001,0.2,30).tolist(),
                'max_depth': np.arange(10,300,50).tolist()}
search_criteria ={'strategy' : "RandomDiscrete",
                        'max_models' : 10, 
                        'seed' : 1}

# Train the grid
xgb_grid = h2o.grid(algorithm = "xgboost",
                     x = x, y = y,
                     training_frame = train,
                     nfolds = 5,
                     seed = 1,
                     hyper_params = hyper_params,
                     )


# Sort the grid by CV AUC
grid <- h2o.getGrid(grid_id = xgb_grid@grid_id, sort_by = "MAE", decreasing = TRUE)
grid_top_model <- grid@summary_table[1, "model_ids"]

TypeError: 'module' object is not callable

In [92]:
h2o.cluster().shutdown()

H2O session _sid_a274 closed.
